In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lgb
from pandasql import sqldf
from sklearn.metrics import mean_squared_error

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_excel('C:\\Users\\osungar\\Desktop\projects\\sales_forecasting\\sales_project\\data\\satis_new.xlsx')


In [4]:
df.head()

,CARI_NO,CARI_ADI,STOK_NO,STOK_KODU,TOPLAM_BIRINCI_OB,OB1,TOPLAM_IKINCI_OB,OB2,SATIS_TARIHI,DVZ_CINSI,IMDSC1,IMSRP1SDC,IMSRP2SDC,IMSRP3SDC,IMSRP4SDC,IMSRP5SDC,IMSRP6SDC,IMSRP7SDC,IMSRP8SDC,IMSRP9SDC,IMSRP0SDC,IMPRP1DSC,IMPRP2DSC,IMPRP3DSC,IMPRP4DSC,IMPRP5DSC,IMPRP6DSC,IMPRP7DSC,IMPRP8DSC,IMPRP9DSC,IMPRP0DSC,IMSHCNDSC,IMSHCMDSC
0,10636,MEKLAS OTOMOTİV İTHALAT İHRACAT SANAYİ,1743,MML.TKN.KAC.MEL.019,303.75,KG,7.000,AD,2015-04-01,USD,"Mek 575mm 0,9mm-NYL-411012-11",TEKNİK HAMUR,0,0,0,0,0,MEKLAS,ŞEKİLLENDİRME,0,0,Mamul,0,0,Ana Urun grubu,0,Karışım Siyah,Parafinik Yağlı,KORUK,0,ASTAR,KALENDER CIKISLI,.
1,10657,VIBRACOUSTIC ÇERKEZKÖY OTOMOTİV SAN.TİC.,743090,MML.TKN.BAC.TRB.124,198.00,KG,1.564,B3,2015-04-01,EUR,Vibracoustic WAR 4500,TEKNİK HAMUR,0,0,0,0,Otomatik İş Emri,TRELLEBORG TBVC BURSA,KARISIM,0,Iki Kademe,Mamul,0,0,Ana Urun grubu,0,Karışım Renkli,Parafinik Yağlı,ANTIVIBRASYON,0,0,BANBURY CIKISLI,.
2,10636,MEKLAS OTOMOTİV İTHALAT İHRACAT SANAYİ,1737,MML.TKN.KAC.MEL.013,1558.50,KG,30.000,AD,2015-04-01,USD,"Mek 665mm 1,2mm-NYL-411042-11",TEKNİK HAMUR,0,0,0,0,0,MEKLAS,ŞEKİLLENDİRME,0,0,Mamul,0,0,Ana Urun grubu,0,Karışım Siyah,Parafinik Yağlı,KORUK,0,ASTAR,KALENDER CIKISLI,.
3,10657,VIBRACOUSTIC ÇERKEZKÖY OTOMOTİV SAN.TİC.,1979,MML.TKN.BAC.TRB.018,268.00,KG,1.000,B3,2015-04-01,EUR,Vibracoustic T 057,TEKNİK HAMUR,0,0,0,0,Otomatik İş Emri,TRELLEBORG TBVC BURSA,KARISIM,0,Üç Kademe,Mamul,0,0,Ana Urun grubu,0,Karışım Renkli,Parafinik + Antilüks 111,ANTIVIBRASYON,0,0,BANBURY CIKISLI,.
4,10636,MEKLAS OTOMOTİV İTHALAT İHRACAT SANAYİ,1736,MML.TKN.KAC.MEL.012,762.00,KG,23.000,AD,2015-04-01,USD,"Mek 370mm 1,4mm-NYL-411028-11",TEKNİK HAMUR,0,0,0,0,0,MEKLAS,ŞEKİLLENDİRME,0,0,Mamul,0,0,Ana Urun grubu,0,Karışım Siyah,Parafinik Yağlı,KORUK,0,ASTAR,KALENDER CIKISLI,.


In [5]:
query = """
SELECT CARI_NO,STOK_NO,TOPLAM_BIRINCI_OB,OB1,TOPLAM_IKINCI_OB,OB2,SATIS_TARIHI                 
FROM df
"""

# Run the query
df_dropped = sqldf(query, locals())
df_dropped.head()

,CARI_NO,STOK_NO,TOPLAM_BIRINCI_OB,OB1,TOPLAM_IKINCI_OB,OB2,SATIS_TARIHI
0,10636,1743,303.75,KG,7.000,AD,2015-04-01 00:00:00.000000
1,10657,743090,198.00,KG,1.564,B3,2015-04-01 00:00:00.000000
2,10636,1737,1558.50,KG,30.000,AD,2015-04-01 00:00:00.000000
3,10657,1979,268.00,KG,1.000,B3,2015-04-01 00:00:00.000000
4,10636,1736,762.00,KG,23.000,AD,2015-04-01 00:00:00.000000


In [6]:
df_dropped.tail()

,CARI_NO,STOK_NO,TOPLAM_BIRINCI_OB,OB1,TOPLAM_IKINCI_OB,OB2,SATIS_TARIHI
243846,55354,756313,7.00,AD,102.15,KG,2023-10-30 00:00:00.000000
243847,10847,2897,51.25,KG,1.00,AD,2023-10-30 00:00:00.000000
243848,10847,2966,249.85,KG,5.00,AD,2023-10-30 00:00:00.000000
243849,10847,2980,115.95,KG,5.00,AD,2023-10-30 00:00:00.000000
243850,10847,2913,48.60,KG,1.00,AD,2023-10-30 00:00:00.000000


In [7]:
df_dropped["SATIS_TARIHI"] = df_dropped["SATIS_TARIHI"].astype("datetime64[ns]")


In [8]:
df_dropped.tail()

,CARI_NO,STOK_NO,TOPLAM_BIRINCI_OB,OB1,TOPLAM_IKINCI_OB,OB2,SATIS_TARIHI
243846,55354,756313,7.00,AD,102.15,KG,2023-10-30
243847,10847,2897,51.25,KG,1.00,AD,2023-10-30
243848,10847,2966,249.85,KG,5.00,AD,2023-10-30
243849,10847,2980,115.95,KG,5.00,AD,2023-10-30
243850,10847,2913,48.60,KG,1.00,AD,2023-10-30


In [9]:
df_dropped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243851 entries, 0 to 243850
Data columns (total 7 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   CARI_NO            243851 non-null  int64         
 1   STOK_NO            243851 non-null  int64         
 2   TOPLAM_BIRINCI_OB  243851 non-null  float64       
 3   OB1                243851 non-null  object        
 4   TOPLAM_IKINCI_OB   243851 non-null  float64       
 5   OB2                243851 non-null  object        
 6   SATIS_TARIHI       243851 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(2), object(2)
memory usage: 13.0+ MB


In [10]:
df_date = df_dropped.copy()

In [11]:
## Kasım 2023'ten önceki veriler
##df_date =df_date[df_date['SATIS_TARIHI'] < '2023-11-01']

In [12]:
df_date['YIL'] = df_date['SATIS_TARIHI'].dt.year
df_date['AY'] = df_date['SATIS_TARIHI'].dt.month

In [13]:
df_date = df_date.drop(columns=['SATIS_TARIHI'])

In [14]:
df_date.tail()

,CARI_NO,STOK_NO,TOPLAM_BIRINCI_OB,OB1,TOPLAM_IKINCI_OB,OB2,YIL,AY
243846,55354,756313,7.00,AD,102.15,KG,2023,10
243847,10847,2897,51.25,KG,1.00,AD,2023,10
243848,10847,2966,249.85,KG,5.00,AD,2023,10
243849,10847,2980,115.95,KG,5.00,AD,2023,10
243850,10847,2913,48.60,KG,1.00,AD,2023,10


In [15]:
df_date_kg = df_date[df_date['OB1']=='KG']

In [16]:
df_date_kg['OB1'].nunique()

1

In [17]:
df_date_kg = df_date_kg.drop(columns=['OB1'])

In [18]:
df_date_kg.tail()

,CARI_NO,STOK_NO,TOPLAM_BIRINCI_OB,TOPLAM_IKINCI_OB,OB2,YIL,AY
243843,10700,754392,2062.80,40.0,AD,2023,10
243847,10847,2897,51.25,1.0,AD,2023,10
243848,10847,2966,249.85,5.0,AD,2023,10
243849,10847,2980,115.95,5.0,AD,2023,10
243850,10847,2913,48.60,1.0,AD,2023,10


# OB1 = KG and OB2 = AD

In [19]:
df_date_kg_ad = df_date_kg[df_date_kg['OB2']=='AD']

In [20]:
df_date_kg_ad = df_date_kg_ad.drop(columns=['OB2'])

In [21]:
df_date_kg_ad.tail()

,CARI_NO,STOK_NO,TOPLAM_BIRINCI_OB,TOPLAM_IKINCI_OB,YIL,AY
243843,10700,754392,2062.80,40.0,2023,10
243847,10847,2897,51.25,1.0,2023,10
243848,10847,2966,249.85,5.0,2023,10
243849,10847,2980,115.95,5.0,2023,10
243850,10847,2913,48.60,1.0,2023,10


# To Month

In [22]:
query = """
SELECT CARI_NO,STOK_NO,AY,YIL,AVG(TOPLAM_IKINCI_OB) as OB2_AYLIK_ORTALAMA,AVG(TOPLAM_BIRINCI_OB) as OB1_AYLIK_ORTALAMA        
FROM df_date_kg_ad
group by CARI_NO,STOK_NO,YIL,AY
order by YIL,AY ASC
"""

# Run the query
df_month_kg_ad = sqldf(query, locals())
df_month_kg_ad.head(10)

,CARI_NO,STOK_NO,AY,YIL,OB2_AYLIK_ORTALAMA,OB1_AYLIK_ORTALAMA
0,10443,4332,4,2015,22.000000,435.425
1,10443,744535,4,2015,15.000000,300.800
2,10563,3049,4,2015,2.000000,61.200
3,10564,2594,4,2015,1.500000,72.600
4,10564,2595,4,2015,1.000000,51.950
5,10564,2602,4,2015,1.000000,46.350
6,10564,2624,4,2015,1.000000,49.400
7,10564,2626,4,2015,1.000000,50.850
8,10564,2628,4,2015,1.333333,72.950
9,10564,2663,4,2015,1.000000,48.550


her bir cari_no 'nun' stok bazında her ay ve yıl için kırılımına bak, olmayan aylara 0 ver. 6*12 aylık veri.
cari_no ,stok_no ay ve yil bazinda sırala.


chat gptye, cari_no stokno bazınıda her bir carinonun stok bazı kırılımında 3 aylık 6 aylık rolling yap.

In [23]:
df_month_kg_ad.tail()

,CARI_NO,STOK_NO,AY,YIL,OB2_AYLIK_ORTALAMA,OB1_AYLIK_ORTALAMA
70921,57210,2922,10,2023,4.0,229.45
70922,57210,2924,10,2023,8.0,485.45
70923,57210,756157,10,2023,2.0,39.85
70924,57439,3089,10,2023,4.0,170.95
70925,57439,3091,10,2023,9.0,407.00


# 

In [24]:
df_month_kg_ad.to_excel('C:\\Users\\osungar\\Desktop\projects\\sales_forecasting\\sales_project\\data\\df_month_kg_ad.xlsx', index=False)


JUMP TO TEST3.1

# TEST1

df_test = df_month_kg_ad.copy()


# CARI_NO ve STOK_NO bazında gruplama
grouped_df = df_month_kg_ad.groupby(['CARI_NO', 'STOK_NO'])

# 3 aylık window rolling
df_test['3_AYLIK_WINDOW'] = grouped_df['OB1_AYLIK_ORTALAMA'].rolling(window=3, min_periods=1).mean().reset_index(level=[0, 1], drop=True)

# 6 aylık window rolling
df_test['6_AYLIK_WINDOW'] = grouped_df['OB1_AYLIK_ORTALAMA'].rolling(window=6, min_periods=1).mean().reset_index(level=[0, 1], drop=True)

# Sonuçları göster
df_test

   CARI_NO  STOK_NO  TOPLAM_BIRINCI_OB  TOPLAM_IKINCI_OB  OB2   YIL  AY  3_AYLIK_WINDOW  6_AYLIK_WINDOW
0        2      101                 5                2    1  2023   4        5.000000        5.000000
1        2      101                 8                4    3  2023   5        6.500000        6.500000
2        5      101                12                6    5  2023   6       12.000000       12.000000
3        5      201                18                8    7  2023   4       15.000000       15.000000
4        5      201                22               10    9  2023   5       20.000000       20.000000
5        5      201                28               12   11  2023   6       22.666667       22.666667
6        3      301                32               14   13  2023   4       32.000000       32.000000
7        3      301                40               16   15  2023   5       36.000000       36.000000
8        3      301                45               18   17  2023   6       39.000000       39.000000


# TEST2

# CARI_NO ve STOK_NO bazında gruplama
grouped_df = df_month_kg_ad.groupby(['CARI_NO', 'STOK_NO'])

# 3 aylık window rolling
df_test['3_AYLIK_WINDOW'] = grouped_df['OB1_AYLIK_ORTALAMA'].rolling(window=3, min_periods=1).mean().shift(1).fillna(0).reset_index(level=[0, 1], drop=True)

# 6 aylık window rolling
df_test['6_AYLIK_WINDOW'] = grouped_df['OB1_AYLIK_ORTALAMA'].rolling(window=6, min_periods=1).mean().shift(1).fillna(0).reset_index(level=[0, 1], drop=True)

# Sonuçları göster
df_test

# TEST3

query = """
SELECT *     
FROM df_test
where YIL=2023
"""

# Run the query
test = sqldf(query, locals())
test

# TEST 3.1

In [30]:
df_test = df_month_kg_ad.copy()


In [26]:

df_test.head(20)

,CARI_NO,STOK_NO,AY,YIL,OB2_AYLIK_ORTALAMA,OB1_AYLIK_ORTALAMA
0,10443,4332,4,2015,22.000000,435.425000
1,10443,744535,4,2015,15.000000,300.800000
2,10563,3049,4,2015,2.000000,61.200000
3,10564,2594,4,2015,1.500000,72.600000
4,10564,2595,4,2015,1.000000,51.950000
5,10564,2602,4,2015,1.000000,46.350000
6,10564,2624,4,2015,1.000000,49.400000
7,10564,2626,4,2015,1.000000,50.850000
8,10564,2628,4,2015,1.333333,72.950000
9,10564,2663,4,2015,1.000000,48.550000


## rolling 2


In [32]:
df_test = df_test.sort_values(by=['CARI_NO', 'STOK_NO', 'YIL', 'AY'])

# 'CARI_NO' ve 'STOK_NO' sütunlarına göre gruplama
grouped = df_test.groupby(['CARI_NO', 'STOK_NO'])



df_test['3_Aylık_Rolling'] = grouped['OB1_AYLIK_ORTALAMA'].rolling(window=3, min_periods=1).mean().reset_index(level=0, drop=True).shift(1).fillna(0)

# 3 aylık pencere (window) oluşturma ve shift(1) uygulama
df['3_AYLIK_ROLLING'] = grouped['OB1_AYLIK_ORTALAMA'].transform(lambda x: x.shift(1).rolling(window=3, min_periods=1).mean())

# NaN değerleri 0 ile doldurma
df['3_AYLIK_ROLLING'] = df['3_AYLIK_ROLLING'].fillna(0)

TypeError: incompatible index of inserted column with frame index

In [ ]:
# MultiIndex oluşturarak 'CARI_NO' ve 'STOK_NO' sütunlarına göre grupla

# Her bir grup için 3 aylık ve 6 aylık hareketli ortalamayı hesapla ve NaN değerlerini 0 ile doldur
df_test['3_AYLIK_ROLLING_OB2'] = grouped_df['OB2_AYLIK_ORTALAMA'].groupby(level=['CARI_NO', 'STOK_NO']).shift(1).rolling(window=3, min_periods=1).mean().reset_index(level=[0, 1], drop=True).fillna(0)
df_test['6_AYLIK_ROLLING_OB2'] = grouped_df['OB2_AYLIK_ORTALAMA'].groupby(level=['CARI_NO', 'STOK_NO']).shift(1).rolling(window=6, min_periods=1).mean().reset_index(level=[0, 1], drop=True).fillna(0)


In [ ]:
df_test.head(100)

In [28]:
df_test.to_excel('C:\\Users\\osungar\\Desktop\projects\\sales_forecasting\\sales_project\\data\\df_rolling.xlsx', index=False)


In [ ]:
10443	4332
10563	2986


In [ ]:
query = """
SELECT *       
FROM df_test
where cari_no = 10847 and stok_no = 4339
order by YIL,AY 
"""

# Run the query
df_count = sqldf(query, locals())
df_count.head(20)

In [ ]:
df_count.to_excel('C:\\Users\\osungar\\Desktop\projects\\sales_forecasting\\sales_project\\data\\df_count.xlsx', index=False)


    cari    stok    count
0	10847	4339	103
1	10847	743463	103
2	10847	2936	101
3	10847	2980	101

## kullanılan window rolling

df['3_AYLIK_ROLLING'] = grouped['OB1_AYLIK_ORTALAMA'].groupby(level=['CARI_NO', 'STOK_NO']).shift(1).rolling(window=3, min_periods=1).mean().reset_index(level=[0, 1], drop=True).fillna(0)


In [ ]:
import pandas as pd

# Örnek DataFrame'i oluşturun (gerçek verilerinizle değiştirin)
data = {'CARI_NO': [1, 1, 1, 2, 2, 3, 3, 3, 3],
        'STOK_NO': [101, 101, 101, 102, 102, 103, 103, 103, 103],
        'AY': [1, 2, 3, 1, 2, 1, 2, 3, 4],
        'YIL': [2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023, 2023],
        'OB1_AYLIK_ORTALAMA': [10, 15, 20, 25, 30, 35, 40, 45, 50]}

df = pd.DataFrame(data)

# MultiIndex oluşturarak 'CARI_NO' ve 'STOK_NO' sütunlarına göre grupla
grouped = df.set_index(['CARI_NO', 'STOK_NO'])

# Her bir grup için 3 aylık ve 6 aylık hareketli ortalamayı hesapla ve NaN değerlerini 0 ile doldur
df['3_AYLIK_ROLLING'] = grouped['OB1_AYLIK_ORTALAMA'].groupby(level=['CARI_NO', 'STOK_NO']).shift(1).rolling(window=3, min_periods=1).mean().reset_index(level=[0, 1], drop=True).fillna(0)
df['6_AYLIK_ROLLING'] = grouped['OB1_AYLIK_ORTALAMA'].groupby(level=['CARI_NO', 'STOK_NO']).shift(1).rolling(window=6, min_periods=1).mean().reset_index(level=[0, 1], drop=True).fillna(0)

print(df)


   CARI_NO  STOK_NO  AY   YIL  OB1_AYLIK_ORTALAMA  3_AYLIK_ROLLING  6_AYLIK_ROLLING
0        1      101   1  2023                  10              0.0              0.0
1        1      101   2  2023                  15             10.0              0.0
2        1      101   3  2023                  20             12.5              0.0
3        1      101   1  2023                  25             15.0              0.0
4        2      102   2  2023                  30              0.0              0.0
5        2      102   1  2023                  35             30.0              0.0
6        3      103   2  2023                  40              0.0              0.0
7        3      103   3  2023                  45             40.0              0.0
8        3      103   4  2023                  50             42.5             32.5


# TRAINING A LGB MODEL

In [ ]:
categorical_columns=['CARI_NO', 'STOK_NO', 'AY']
for col in categorical_columns:
    df_test[col] = df_test[col].astype('category')

In [ ]:
df_test.info()

In [ ]:
X = df_test.drop(columns=['OB1_AYLIK_ORTALAMA','OB2_AYLIK_ORTALAMA'])

y = df_test['OB1_AYLIK_ORTALAMA']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [ ]:
# create dataset for lightgbm
lgb_train = lgb.Dataset(X_train, y_train,categorical_feature=categorical_columns)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

In [ ]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'random_state':42
}

In [ ]:
# Light Gradient Boosting Regressor
lgb_trained =lgb.train(params,
                lgb_train)


In [ ]:
y_pred =lgb_trained.predict(X_test)


In [ ]:
print('MSE score on train data:')
print(mean_squared_error(y_test,y_pred))

In [ ]:
merged_array = np.stack((y_test, y_pred), axis=1)


In [ ]:
print("         y_test      ,      y_pred")
merged_array[:50]

# PREDICTING 11/2023

## Creating dump df

In [ ]:
query = """
SELECT DISTINCT CARI_NO,STOK_NO     
FROM df_test
"""

# Run the query
df_dump = sqldf(query, locals())
df_dump.head(10)

In [ ]:
df_test.columns

In [ ]:
df_dump["AY"] = 11
df_dump["YIL"] = 2023
df_dump["OB2_AYLIK_ORTALAMA"] = 1
df_dump["OB1_AYLIK_ORTALAMA"] = 1


In [ ]:
df_dump.head()

In [ ]:
merge1= df_month_kg_ad.copy()

In [ ]:
result_df = pd.concat([merge1, df_dump], axis=0, ignore_index=True)


In [ ]:
result_df.head()

In [ ]:
query = """
SELECT CARI_NO,STOK_NO,AY,YIL,OB2_AYLIK_ORTALAMA,OB1_AYLIK_ORTALAMA     
FROM result_df
group by CARI_NO,STOK_NO,YIL,AY
order by YIL,AY ASC
"""

# Run the query
result_df = sqldf(query, locals())
result_df.head(10)

In [ ]:
result_df.tail()

In [ ]:
# MultiIndex oluşturarak 'CARI_NO' ve 'STOK_NO' sütunlarına göre grupla
grouped_df = result_df.set_index(['CARI_NO', 'STOK_NO'])

# Her bir grup için 3 aylık ve 6 aylık hareketli ortalamayı hesapla ve NaN değerlerini 0 ile doldur
result_df['3_AYLIK_ROLLING_OB1'] = grouped_df['OB1_AYLIK_ORTALAMA'].groupby(level=['CARI_NO', 'STOK_NO']).shift(1).rolling(window=4, min_periods=1).mean().reset_index(level=[0, 1], drop=True).fillna(0)
result_df['6_AYLIK_ROLLING_OB1'] = grouped_df['OB1_AYLIK_ORTALAMA'].groupby(level=['CARI_NO', 'STOK_NO']).shift(1).rolling(window=7, min_periods=1).mean().reset_index(level=[0, 1], drop=True).fillna(0)

result_df['3_AYLIK_ROLLING_OB2'] = grouped_df['OB2_AYLIK_ORTALAMA'].groupby(level=['CARI_NO', 'STOK_NO']).shift(1).rolling(window=4, min_periods=1).mean().reset_index(level=[0, 1], drop=True).fillna(0)
result_df['6_AYLIK_ROLLING_OB2'] = grouped_df['OB2_AYLIK_ORTALAMA'].groupby(level=['CARI_NO', 'STOK_NO']).shift(1).rolling(window=7, min_periods=1).mean().reset_index(level=[0, 1], drop=True).fillna(0)


In [ ]:
query = """
SELECT *     
FROM result_df
where AY=11 and YIL=2023
"""

# Run the query
last = sqldf(query, locals())
last.head(10)

In [ ]:
last = last.drop(columns=['OB1_AYLIK_ORTALAMA','OB2_AYLIK_ORTALAMA'])


In [ ]:
last.head(10)

In [ ]:
last.info()

In [ ]:
categorical_columns=['CARI_NO', 'STOK_NO', 'AY']
for col in categorical_columns:
    last[col] = last[col].astype('category')

In [ ]:
y_pred_last =lgb_trained.predict(last)


In [ ]:
y_pred_last

In [ ]:
last["PREDICTION"]= y_pred_last

In [ ]:
df_test_prediction = df_date[(df_date['AY'] == 9) & (df_date['YIL'] == 2023)]

In [ ]:
df_test_prediction.columns

In [ ]:
last["PREDICTION_PREV"] = df_test_prediction['TOPLAM_BIRINCI_OB']

In [ ]:
last

In [ ]:
print('MSE score on train data:')
print(mean_squared_error(y_test,y_pred))